In [34]:
import pandas as pd
import numpy as np

# ---------------------------------------------METHODS-----------------------------------------------------

def data_research(df_list:list , column_names = []):
    for df in df_list:
        if len(column_names) == 0:
            columns = df.columns
        else:
            columns = column_names
        for c in columns:
            nan = 0
            l = len(df[c])
            for n in df[c]:
                if str(n).lower() == 'nan':
                    nan += 1
            print(f'{c} all:{l} unique:{len(df[c].dropna().unique())} nan:{nan}')
        print('\n')

def get_nans(df, column_name):
    return df.index[df[column_name]=='nan'].to_list()

def levenstein(a, b):
    """ This funciton return minimal count of changes to
        lead "a" to "b"
    """
    f = [[a+b if a*b==0 else 0 for b in range(len(b))] for a in range(len(a))]
    for i in range(1, len(a)):
        for j in range(1, len(b)):
            if a[i-1] == b[j-1]:
                f[i][j] = f[i-1][j-1]
            else:
                f[i][j] = 1 + min(f[i-1][j], f[i][j-1], f[i-1][j-1])
    return f[-1][-1]

# ----------------------------------------------------------------------------------------------------------

# READ FILE and EXPORT TO DATA FRAMES
path = '/Users/ivankudravcev/Studying/Data/aspex.xlsx'
xls = pd.ExcelFile(path)
sheets = xls.sheet_names

data = {sht:pd.read_excel(xls, sheet_name=sht) for sht in sheets}

# DATA REVIEW (be care: too many tabels)
"""
for k, v in data.items():
    print(k, 'records:',len(v), '\n')
    print(v.head())
    print(2*'\n') 
"""

# SEPARATE DATA FRAMES
sales = data['Продажи']
costs = data['CC']
gen = pd.DataFrame()
for sheet in sheets[:-3]:
    gen = gen.append(data[sheet])
    
# RENAME SAME COLUMNS
sales.rename(columns = {'Цена со скидкой (фактическая цена продажи)':'Продажа', 'Рекомендуемая цена':'Розница'}, inplace = True)
costs.rename(columns = {'СС - тенге':'Себестоимость'}, inplace = True)

# CHANGE SAME DATA TYPES
sales['Наименование'] = pd.DataFrame(sales['Наименование'], dtype='str')
costs['Наименование'] = pd.DataFrame(costs['Наименование'], dtype='str')
sales['Себестоимость'] = pd.DataFrame(sales['Себестоимость'], dtype='str')

# QUICK REVIEW 
print('"SALES" BEFORE')
data_research([sales])

# GET INDICES WITH NAN
ind_nan_seb = get_nans(sales, 'Себестоимость')

# GET DICT KEY(NAME FROM DF#1):VALUES(SIMILAR NAMES FROM DF#2)
dict1 = {}
for n1 in sales.iloc[ind_nan_seb]['Наименование']:
    if n1 != 'nan':
        dict1.setdefault(n1, [])
        for n2 in costs['Наименование']:
            if n2.lower().replace(' ','').startswith(n1.lower().replace(' ','')[:3]):
                dict1[n1].append(n2)
    
# GET NEW DICT: CHOOSE 1 CLOSEST NAME FOR KEY FROM VALUES IN DICT
dict2 = {}
checked = []
for key, val in dict1.items():
    if key not in checked:
        checked.append(key)
        distance = np.infty
        for v in val:
            d = levenstein(key.lower().replace(' ',''), v.lower().replace(' ',''))
            if d < distance:
                distance = d
                dict2[key]=v

# FILL NAN VALUES USING NEW DICT
for k, v in dict2.items():
    non_value = costs[costs['Наименование']==v]['Себестоимость'].values[0]
    value = costs[costs['Наименование']==v]['Розница'].values[0]
    sales.loc[sales.Наименование==k, 'Себестоимость'] = non_value
    sales.loc[sales.Наименование==k, 'Розница'] = value


# QUICK REVIEW
print('"SALES" AFTER')
data_research([sales])

print('done')

"SALES" BEFORE
Число all:6326 unique:31 nan:1
Месяц all:6326 unique:5 nan:1
Наименование all:6326 unique:2221 nan:5
Розница all:6326 unique:85 nan:795
Продажа all:6326 unique:487 nan:1
Имя продавца all:6326 unique:20 nan:1078
Вид товара all:6326 unique:9 nan:119
Себестоимость all:6326 unique:43 nan:795
Валовая прибыль all:6326 unique:557 nan:795


"SALES" AFTER
Число all:6326 unique:31 nan:1
Месяц all:6326 unique:5 nan:1
Наименование all:6326 unique:2221 nan:5
Розница all:6326 unique:116 nan:23
Продажа all:6326 unique:487 nan:1
Имя продавца all:6326 unique:20 nan:1078
Вид товара all:6326 unique:9 nan:119
Себестоимость all:6326 unique:78 nan:23
Валовая прибыль all:6326 unique:557 nan:795


done


In [35]:
ind_nan_prib = get_nans(sales,'Себестоимость')
len(ind_nan_prib)

sales.iloc[ind_nan_prib]

,Число,Месяц,Наименование,Розница,Продажа,Имя продавца,Вид товара,Себестоимость,Валовая прибыль
139,7.0,January,nan,NaN,0.0,NaN,NaN,nan,NaN
140,7.0,January,nan,NaN,0.0,NaN,NaN,nan,NaN
758,29.0,January,Джозеппи Заноти кожанный,NaN,20720.0,Каракоз,Обувь,nan,NaN
978,4.0,February,Аксессуары по 1000,NaN,500.0,Каракоз,Аксессуары,nan,NaN
1504,18.0,February,Аксессуары по 1000,NaN,500.0,Акира,Аксессуары,nan,NaN
2072,28.0,February,гольфы Pirex 23,NaN,2199.0,Акира,Аксессуары,nan,NaN
2156,1.0,March,Гольфы Pyrex 23,NaN,1980.0,Акира,Аксессуары,nan,NaN
2775,13.0,March,Гольфы Pyrex,NaN,2200.0,Акмарал,Аксессуары,nan,NaN
2776,13.0,March,Гольфы Pyrex,NaN,2200.0,Акмарал,Аксессуары,nan,NaN
3416,25.0,March,nan,NaN,15200.0,Каракоз,Обувь,nan,NaN


In [36]:
costs['Категория'].unique()

array(['Зимняя мужская верхняя одежда', 'Обувь Для скейтбординга',
       'Обувь Зимняя', 'Обувь Повседневная', 'Обувь Для стиля',
       'Обувь Баскетбольная', 'мужская прочая одежда', 'прочее', 'доски',
       'мужская верхняя одежда', 'Обувь Casual', 'прочий скейт стафф',
       'Long boards', 'Обувь Беговая', 'Футбольные бутсы', 'Penny boards',
       'снэпы', nan, 'женская верхняя одежда', 'Одежда Bat Norton',
       'женская прочая одежда', 'браслеты', 'прочие аксессуары',
       'мужские джинсы', 'зимние аксессуары', 'Носки',
       'женские аксессуары', 'Salamander', 'женские джинсы',
       'женские шорты/юбки', 'Зимняя женская верхняя одежда',
       'прочие головные уборы', 'наушники', 'нижнее белье', 'носки',
       'очки', 'сумки/пеналы', 'мужские шорты', 'подвески', 'портмоне',
       'ремни', 'рубашки', 'свитшоты/свитера', 'тетради ',
       'часы (закуп у 4u)', 'чехлы на телефон', 'шапки'], dtype=object)

In [37]:
# FOUND SOCKS
costs[costs.Категория=='Носки']

,Наименование,Категория,Себестоимость,Розница,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
1463,Носки Pyrex/black,Носки,450.0,2199.0,NaN,NaN,NaN,NaN
1464,Носки Pyrex/navy blue,Носки,450.0,2199.0,NaN,NaN,NaN,NaN
1465,Носки Pyrex/red,Носки,450.0,2199.0,NaN,NaN,NaN,NaN
1466,Носки Pyrex/white,Носки,450.0,2199.0,NaN,NaN,NaN,NaN
1467,Носки Pyrex/yellow,Носки,450.0,2199.0,NaN,NaN,NaN,NaN


In [128]:
socks = [2072, 2156, 2775, 2776, 3769, 3983, 4400, 5261, 5262, 5872, 6044, 6105, 6149, 6204]

non_value = 450
value = 2199
sales.loc[:,'Себестоимость'].iloc[socks] = non_value
sales.loc[:,'Розница'].iloc[socks] = value

ind = get_nans(sales,'Себестоимость')
print(sales.iloc[ind])

      Число    Месяц Наименование  Розница  Продажа Имя продавца Вид товара  \
139     7.0  January          nan      NaN      0.0          NaN        NaN   
140     7.0  January          nan      NaN      0.0          NaN        NaN   
3683    NaN      NaN          nan      NaN      NaN          NaN          0   

     Себестоимость  Валовая прибыль  
139            nan              NaN  
140            nan              NaN  
3683           nan              NaN  


"\n# CHANGE SAME DATA TYPES\nsales['Себестоимость'] = pd.DataFrame(sales['Себестоимость'], dtype='float')\n\n# CALCULATE 'Валовая прибыль'\nsales['Валовая прибыль'] = sales['Продажа']-sales['Себестоимость'] \n"

In [ ]:
# FIND ANOTHERS
gen[gen.Наименование=='Джозеппи Заноти кожанный']
gen[gen.Наименование=='Кабез women /ragged']
gen.loc[(gen.Месяц=='March') & (gen.Число==25) & (gen['Цена со скидкой']==15200)]
gen.loc[(gen.Месяц=='May') & (gen.Число==30) & (gen['Цена со скидкой']==11950)]
gen[gen.Наименование=='Аксессуары по 1000']

In [127]:
# Joseppe...
costs[costs['Розница']==20720]
sales.loc[:,'Себестоимость'].iloc[758] = 9500

# Кабез
pd.options.display.max_rows = 100
costs[costs['Розница']==6950]
sales.loc[:,'Себестоимость'].iloc[4006] = 2800

# 15200 (???)
cost = costs[costs['Розница']==15200]['Себестоимость'].mean()
sales.loc[:,'Себестоимость'].iloc[3416] = cost

# 11950 (Converse?)
cost = costs[costs['Розница']==11950]['Себестоимость'].mean()
sales.loc[:,'Себестоимость'].iloc[6274] = cost

# Аксессуары по 1000
costs[costs['Розница']==1000]
sales.loc[:,'Себестоимость'].iloc[978] = 400
sales.loc[:,'Себестоимость'].iloc[1504] = 500

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [129]:
# CHANGE SAME DATA TYPES
sales['Себестоимость'] = pd.DataFrame(sales['Себестоимость'], dtype='float')

# CALCULATE 'Валовая прибыль'
sales['Валовая прибыль'] = sales['Продажа']-sales['Себестоимость'] 

print('"SALES" AFTER')
data_research([sales])

"SALES" AFTER
Число all:6326 unique:31 nan:1
Месяц all:6326 unique:5 nan:1
Наименование all:6326 unique:2221 nan:5
Розница all:6326 unique:117 nan:9
Продажа all:6326 unique:487 nan:1
Имя продавца all:6326 unique:20 nan:1078
Вид товара all:6326 unique:9 nan:119
Себестоимость all:6326 unique:70 nan:3
Валовая прибыль all:6326 unique:664 nan:3




In [130]:
# Save data
sales.to_excel('/Users/ivankudravcev/Studying/Data/sales_filled.xlsx')